Quiero comparar modelos de sentiment analysis que están disponibles en Hugging face. 
Uso este tutorial:
https://huggingface.co/blog/sentiment-analysis-python


In [1]:
#pip install -q transformers

In [1]:
#pip install transformers==4.28.0

In [2]:
#pip install torch

In [2]:
from datasets import load_dataset
import pandas as pd

Probando el uso de un modelo pre-existente

In [3]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

Defino un modelo en específico: bertweet-base-sentiment-analysis

In [4]:

specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model(data)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

otros textos

In [6]:
data = ["te quiero", "el dia es horrible"]
specific_model(data)



[{'label': 'POS', 'score': 0.6898192167282104},
 {'label': 'NEG', 'score': 0.9758785367012024}]

Sigo con el ejercicio. Voy a hacer fine tuning de un modelo  

El primer paso es usar un GPU en lugar de CPU pero no se hacerlo de manera local (el ejercicio está previsto para googlecollab)

In [1]:
import torch
torch.cuda.is_available()


False

In [2]:
!pip install datasets transformers huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!apt-get install git-lfs

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Preprocess data  

Bajo la base IMDB de reviews de peliculas  


In [4]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Se extrae una parte para que sea mas chico el ds  


In [5]:
small_train_dataset = imdb["train"].shuffle(seed=14).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=14).select([i for i in list(range(300))])

Se usa el tokenizador de distibert 

The DistilBERT model was proposed in the blog post Smaller, faster, cheaper, lighter: Introducing DistilBERT, a distilled version of BERT, and the paper DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter. DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
small_train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})

los features son Text y label. 

Aqui aplico el tokenizador al texto de cada dataset, train y test.


tokenized_train = tokenizer(small_train_dataset["text"], truncation=True)

tokenized_test = tokenizer(small_test_dataset["text"], truncation=True)

In [23]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Preprocess

Before you can train a model on a dataset, it needs to be preprocessed into the expected model input format. Whether your data is text, images, or audio, they need to be converted and assembled into batches of tensors. 🤗 Transformers provides a set of preprocessing classes to help prepare your data for the model. In this tutorial, you’ll learn that for:

Text, use a Tokenizer to convert text into a sequence of tokens, create a numerical representation of the tokens, and assemble them into tensors

Pad  

Sentences aren’t always the same length which can be an issue because tensors, the model inputs, need to have a uniform shape. Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to shorter sentences.  


Set the padding parameter to True to pad the shorter sequences in the batch to match the longest sequence:

In [24]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training the model  

In [25]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

Defino una función para computar las métricas del modelo (divino!)

In [26]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

Preparo los argumentos para entrenar el modelo, pero no lo conecto con hugging face sino que lo realizo de manera local

In [27]:
from transformers import TrainingArguments, Trainer
 
#repo_name = "finetuning-sentiment-model-3000-samples"
 
training_args = TrainingArguments(
   output_dir="./output-models",
   #overwrite_output_dir=True,

   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=False,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


In [28]:
trainer.train()

  0%|          | 0/376 [00:00<?, ?it/s]